In [1]:
# --- 1. 라이브러리 및 모듈 임포트 ---

# 데이터 처리 라이브러리
import pandas as pd
import numpy as np

# 데이터 시각화 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns

# 머신러닝 모델 및 평가 도구
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import joblib

# 모델 해석 및 시각화 (SHAP)
# !pip install shap
import shap
from IPython.display import display

# 시스템 및 경로 관련 라이브러리
import sys
import os
from datetime import datetime
import warnings

# 경고 메시지 무시
warnings.filterwarnings('ignore')

import matplotlib.font_manager as fm

try:
    fe = fm.FontEntry(
        fname=r'../../font/NanumFont/NanumGothic.ttf',
        name='NanumGothic')
    fm.fontManager.ttflist.insert(0, fe)
    plt.rcParams.update({'font.size': 10, 'font.family': 'NanumGothic'})
    plt.rc('font', family='NanumGothic')
    
except FileNotFoundError:
    print("나눔고딕 폰트를 찾을 수 없어 기본 폰트로 설정됩니다.")
    pass

In [2]:
# --- 2. 경로 설정 및 커스텀 로거 임포트 ---

try:
    # 현재 노트북 파일의 위치를 기준으로 프로젝트 루트 경로를 설정합니다.
    src_path = os.path.abspath(os.path.join(os.getcwd(), "../../src/log"))
    sys.path.insert(0, src_path)
    from logger import Logger
    print("Logger 모듈 로드 성공.")
    
except ImportError:
    print("[오류] Logger 모듈을 찾을 수 없습니다. src/log/logger.py 경로를 확인해주세요.")
    # Logger가 없어도 코드가 중단되지 않도록 임시 클래스 정의
    class Logger:
        def __init__(self, *args, **kwargs): pass
        def write(self, message, **kwargs): print(message)

Logger 모듈 로드 성공.


In [3]:
# --- 3. 로거 및 경로 초기화 ---

# 로그 파일이 저장될 경로 설정
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
LOG_DIR         = '../../data/logs/price_prediction_logs'
LOG_FILENAME    = f"price_prediction_{timestamp}.log"
LOG_PATH        = os.path.join(LOG_DIR, LOG_FILENAME)

# Directory 생성
os.makedirs(LOG_DIR, exist_ok=True)

# Logger 인스턴스 생성
logger = Logger(log_path=LOG_PATH)

# 데이터 및 제출 파일 경로 설정
SUBMISSION_DIR = '../../data/processed/submissions'
SUBMISSION_FILENAME = 'price_prediction_submission.csv'
SUBMISSION_PATH = os.path.join(SUBMISSION_DIR, SUBMISSION_FILENAME)

# model 저장 경로 설정
MODEL_DIR = '../../model'
MODEL_FILENAME = 'price_prediction_model_1.pkl'
MODEL_PATH = os.path.join(MODEL_DIR, MODEL_FILENAME)

logger.write("="*50)
logger.write(">> 아파트 가격 예측 모델링 시작")

2025-07-15 23:37:24 | ==================================================
2025-07-15 23:37:24 | >> 아파트 가격 예측 모델링 시작


In [4]:
# --- 4. 데이터 로드 (공통) ---

logger.write(">> 정제된 Train/Test 데이터를 로드합니다...")

try:
    # 학습 데이터와 테스트 데이터를 각각 로드
    train_df_clean = pd.read_csv('../../data/processed/cleaned_data/train_clean.csv')
    test_df_clean = pd.read_csv('../../data/processed/cleaned_data/test_clean.csv')
    logger.write(">> Train/Test 데이터 로드 완료.")
    
    # 두 데이터를 합치기 전에 isTest 컬럼으로 구분
    train_df_clean['isTest'] = 0
    test_df_clean['isTest'] = 1
    
    # 하나의 데이터프레임으로 합침
    df = pd.concat([train_df_clean, test_df_clean])
    
    logger.write(">> 데이터 로드 및 병합 완료.")
    
except FileNotFoundError:
    logger.write(f">> [오류] train_clean.csv 또는 test_clean.csv 파일을 찾을 수 없습니다.", print_error=True)
    df = None

2025-07-15 23:37:24 | >> 정제된 Train/Test 데이터를 로드합니다...
2025-07-15 23:37:26 | >> Train/Test 데이터 로드 완료.
2025-07-15 23:37:26 | >> 데이터 로드 및 병합 완료.


In [5]:
# --- 5. 고급 피처 엔지니어링 및 데이터 분리 (성능 개선 버전) ---

if 'df' in locals() and df is not None:
    try:
        logger.write(">> [5단계 시작] 고급 피처 엔지니어링 및 데이터 분리를 시작합니다...")
        
        # 1. Train/Test 데이터 분리 (가장 먼저 수행)
        train_df = df[df['isTest'] == 0].copy()
        test_df = df[df['isTest'] == 1].copy()
        logger.write(">> 1. Train/Test 데이터 분리 완료.")
        
        # 2. 시간 관련 파생변수: 월(Month)의 주기성을 모델에 알려줍니다.
        logger.write(">> 2. 시간의 주기성(Cyclical) 피처 생성 중...")
        for temp_df in [train_df, test_df]:
            temp_df['계약월_sin'] = np.sin(2 * np.pi * temp_df['계약월']/12)
            temp_df['계약월_cos'] = np.cos(2 * np.pi * temp_df['계약월']/12)
        logger.write(">> '계약월_sin', '계약월_cos' 피처 생성 완료.")

        # 3. 면적당 가격(평당가 개념) 파생변수
        logger.write(">> 3. 면적당 가격 관련 피처 생성 중 (데이터 누수 방지)...")
        # 훈련 데이터에만 '면적당가격' 피처를 계산하여 데이터 누수 방지
        train_df['면적당가격'] = train_df['target'] / train_df['전용면적']
        
        # 4. 법정동별 평균 면적당 가격
        dong_price_stats = train_df.groupby('법정동')['면적당가격'].agg(['mean']).reset_index()
        dong_price_stats.columns = ['법정동', '동별_평균면적당가격']
        
        # 5. 훈련 및 테스트 데이터에 병합
        train_df = pd.merge(train_df, dong_price_stats, on='법정동', how='left')
        test_df = pd.merge(test_df, dong_price_stats, on='법정동', how='left')
        logger.write(">> '동별_평균면적당가격' 피처 생성 및 병합 완료.")
        
        # 6. 통계 피처 생성 (데이터 누수 방지)
        logger.write(">> 4. '자치구' 기반 통계 피처 생성 중...")
        sigungu_stats = train_df.groupby('자치구')['target'].agg(['mean', 'std']).reset_index()
        sigungu_stats.columns = ['자치구', '자치구별_평균가격', '자치구별_가격편차']
        
        train_df = pd.merge(train_df, sigungu_stats, on='자치구', how='left')
        test_df = pd.merge(test_df, sigungu_stats, on='자치구', how='left')
        logger.write(">> '자치구' 기반 통계 피처 병합 완료.")

        # 7. 상호작용 특성 생성
        logger.write(">> 5. 상호작용 특성 생성 중...")
        for temp_df in [train_df, test_df]:
            temp_df['면적_x_연식'] = temp_df['전용면적'] * temp_df['연식']
            temp_df['면적_x_층'] = temp_df['전용면적'] * temp_df['층']
        logger.write(">> 상호작용 특성 생성 완료.")

        # --- ✨ 피처 정리 및 최종 데이터셋 생성 ✨ ---
        
        # 8. 테스트 데이터의 결측치(NA) 처리 (훈련 데이터의 전체 평균값으로)
        logger.write(">> 6. 테스트 데이터의 결측치(NA) 처리 중...")
        fill_na_cols = ['동별_평균면적당가격', '자치구별_평균가격', '자치구별_가격편차']
        mean_vals = train_df[fill_na_cols].mean()
        test_df.fillna(mean_vals, inplace=True)
        logger.write(">> 테스트 데이터 결측치 처리 완료.")
        
        # 9. 피처(X)와 타겟(y) 최종 정의
        logger.write(">> 7. 학습에 사용할 피처(X)와 타겟(y) 정의 중...")
        # '면적당가격'은 target으로 만든 변수이므로 학습 피처에서 제외
        features = [col for col in train_df.columns if col not in ['target', 'id', '아파트이름', 'isTest', '면적당가격']]
        
        X_train_raw = train_df[features]
        y_train = train_df['target']
        X_test_raw = test_df[features]
        logger.write(">> 피처/타겟 정의 완료.")

        # 10. 범주형 피처 처리 (Label Encoding)
        logger.write(">> 8. 범주형 피처 Label Encoding 처리 중...")
        categorical_features = X_train_raw.select_dtypes(include=['object']).columns.tolist()
        
        for col in categorical_features:
            le = LabelEncoder()
            all_vals = pd.concat([X_train_raw[col], X_test_raw[col]]).astype(str).unique()
            le.fit(all_vals)
            X_train_raw[col] = le.transform(X_train_raw[col].astype(str))
            X_test_raw[col] = le.transform(X_test_raw[col].astype(str))
            
        logger.write(">> [5단계 완료] 모든 피처 엔지니어링 및 데이터 분리 성공.")

    except Exception as e:
        logger.write(f">> [오류] 피처 엔지니어링 중 심각한 문제 발생: {e}", print_error=True)

2025-07-15 23:37:26 | >> [5단계 시작] 고급 피처 엔지니어링 및 데이터 분리를 시작합니다...
2025-07-15 23:37:26 | >> 1. Train/Test 데이터 분리 완료.
2025-07-15 23:37:26 | >> 2. 시간의 주기성(Cyclical) 피처 생성 중...
2025-07-15 23:37:26 | >> '계약월_sin', '계약월_cos' 피처 생성 완료.
2025-07-15 23:37:26 | >> 3. 면적당 가격 관련 피처 생성 중 (데이터 누수 방지)...
2025-07-15 23:37:26 | >> '동별_평균면적당가격' 피처 생성 및 병합 완료.
2025-07-15 23:37:26 | >> 4. '자치구' 기반 통계 피처 생성 중...
2025-07-15 23:37:27 | >> '자치구' 기반 통계 피처 병합 완료.
2025-07-15 23:37:27 | >> 5. 상호작용 특성 생성 중...
2025-07-15 23:37:27 | >> 상호작용 특성 생성 완료.
2025-07-15 23:37:27 | >> 6. 테스트 데이터의 결측치(NA) 처리 중...
2025-07-15 23:37:27 | >> 테스트 데이터 결측치 처리 완료.
2025-07-15 23:37:27 | >> 7. 학습에 사용할 피처(X)와 타겟(y) 정의 중...
2025-07-15 23:37:27 | >> 피처/타겟 정의 완료.
2025-07-15 23:37:27 | >> 8. 범주형 피처 Label Encoding 처리 중...
2025-07-15 23:37:27 | >> [5단계 완료] 모든 피처 엔지니어링 및 데이터 분리 성공.


In [ ]:
# --- 6. 빠르고 효율적인 모델 학습 (K-Fold CV) ---

if 'X_train_raw' in locals():
    logger.start_redirect()
    
    try:
        logger.write(">> [9단계 시작] K-Fold 교차 검증 및 모델 학습을 시작합니다...")
        
        # 검증된 고성능 하이퍼파라미터 사용 (RandomizedSearchCV 삭제)
        best_params = {
            'objective': 'regression_l1', # MAE
            'metric': 'rmse',
            'n_estimators': 2000,
            'learning_rate': 0.01,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'bagging_freq': 1,
            'lambda_l1': 0.1,
            'lambda_l2': 0.1,
            'num_leaves': 31,
            'verbose': -1,
            'n_jobs': -1,
            'seed': 42,
            'boosting_type': 'gbdt',
        }
        logger.write(f">> 고정 하이퍼파라미터 사용: {best_params}")

        # K-Fold 교차 검증으로 최종 모델 성능 평가 및 예측
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        oof_preds = np.zeros(X_train_raw.shape[0])
        test_preds = np.zeros(X_test_raw.shape[0])
        rmse_scores = []
        models = []
        logger.write(">> K-Fold 설정 완료. 5-Fold 교차 검증을 시작합니다.")

        for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_raw, y_train)):
            logger.write(f"--- [Fold {fold+1}/5] 학습 시작 ---")
            
            logger.write(f">> Fold {fold+1}: 데이터 분할 중...")
            X_train_fold, X_val_fold = X_train_raw.iloc[train_idx], X_train_raw.iloc[val_idx]
            y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
            logger.write(f">> Fold {fold+1}: Train size={len(X_train_fold)}, Validation size={len(X_val_fold)}")

            logger.write(f">> Fold {fold+1}: 모델 생성 및 학습(fit) 시작...")
            model = lgb.LGBMRegressor(device='cuda', **best_params)
            model.fit(X_train_fold, y_train_fold,
                      eval_set=[(X_val_fold, y_val_fold)],
                      eval_metric='rmse',
                      callbacks=[lgb.early_stopping(100, verbose=False)])
            logger.write(f">> Fold {fold+1}: 모델 학습 완료.")

            logger.write(f">> Fold {fold+1}: 검증 데이터 예측(predict) 및 성능 평가 시작...")
            val_preds = model.predict(X_val_fold)
            oof_preds[val_idx] = val_preds
            
            rmse = np.sqrt(mean_squared_error(y_val_fold, val_preds))
            rmse_scores.append(rmse)
            models.append(model)
            logger.write(f"✅ Fold {fold+1} RMSE: {rmse:.4f}")
            
            logger.write(f">> Fold {fold+1}: 테스트 데이터 예측 누적 중...")
            test_preds += model.predict(X_test_raw) / kf.get_n_splits()
            logger.write(f"--- [Fold {fold+1}/5] 처리 완료 ---\n")

        avg_rmse = np.mean(rmse_scores)
        logger.write("-------------------------------------------")
        logger.write(f"✅ 최종 CV 평균 RMSE: {avg_rmse:.4f}")
        logger.write("-------------------------------------------")
        
        # 전체 데이터로 최종 모델 학습
        logger.write(">> 전체 훈련 데이터로 최종 모델 학습 시작...")
        final_model = lgb.LGBMRegressor(device='cuda', **best_params)
        final_model.fit(X_train_raw, y_train)
        logger.write(">> 최종 모델 학습 완료.")
        logger.write(">> [9단계 완료] 모든 모델 학습 및 평가 성공.")

    except Exception as e:
        logger.write(f">> [오류] 모델 학습 중 심각한 문제 발생: {e}", print_error=True)
        
    finally:
        logger.stop_redirect()

2025-07-15 23:37:27 | >> 표준 출력 및 오류를 로그 파일로 리디렉션 시작
2025-07-15 23:37:27 | >> [9단계 시작] K-Fold 교차 검증 및 모델 학습을 시작합니다...
2025-07-15 23:37:27 | >> 고정 하이퍼파라미터 사용: {'objective': 'regression_l1', 'metric': 'rmse', 'n_estimators': 2000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'lambda_l1': 0.1, 'lambda_l2': 0.1, 'num_leaves': 31, 'verbose': -1, 'n_jobs': -1, 'seed': 42, 'boosting_type': 'gbdt'}
2025-07-15 23:37:27 | >> K-Fold 설정 완료. 5-Fold 교차 검증을 시작합니다.
2025-07-15 23:37:27 | --- [Fold 1/5] 학습 시작 ---
2025-07-15 23:37:27 | >> Fold 1: 데이터 분할 중...
2025-07-15 23:37:27 | >> Fold 1: Train size=888080, Validation size=222021
2025-07-15 23:37:27 | >> Fold 1: 모델 생성 및 학습(fit) 시작...
2025-07-15 23:39:07 | >> Fold 1: 모델 학습 완료.
2025-07-15 23:39:07 | >> Fold 1: 검증 데이터 예측(predict) 및 성능 평가 시작...
2025-07-15 23:39:08 | ✅ Fold 1 RMSE: 14041.6938
2025-07-15 23:39:08 | >> Fold 1: 테스트 데이터 예측 누적 중...
2025-07-15 23:39:08 | --- [Fold 1/5] 처리 완료 ---
2025-07-15 23:39:08 | 

In [7]:
# --- 7. 최종 모델 결과 시각화 및 분석 ---

if 'models' in locals() and 'oof_preds' in locals():
    logger.write(">> [7단계 시작] 모델 결과 시각화 및 분석을 시작합니다...")

    # 1. 피처 중요도(Feature Importance) 시각화 (폴드별 평균)
    feature_importances = pd.DataFrame()
    for i, model in enumerate(models):
        fold_importance = pd.DataFrame({
            'feature': X_train_raw.columns,
            'importance': model.feature_importances_,
            'fold': i + 1
        })
        feature_importances = pd.concat([feature_importances, fold_importance], axis=0)
    
    mean_importances = feature_importances.groupby('feature')['importance'].mean().sort_values(ascending=False)
    
    plt.figure(figsize=(12, 10))
    sns.barplot(x=mean_importances.head(20).values, y=mean_importances.head(20).index)
    plt.title('상위 20개 피처 중요도 (평균)', fontsize=16)
    plt.xlabel('중요도', fontsize=12)
    plt.ylabel('피처', fontsize=12)
    plt.grid(True, axis='x', linestyle='--')
    plt.tight_layout()
    plt.show()

    # 2. 실제 값 vs OOF 예측 값 비교 (Out-of-Fold)
    plt.figure(figsize=(10, 10))
    sns.scatterplot(x=y_train, y=oof_preds, alpha=0.3)
    plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', lw=2, label='이상적인 예측선 (y=x)')
    plt.title('실제 값 vs OOF 예측 값 비교', fontsize=16)
    plt.xlabel('실제 값 (Actual)', fontsize=12)
    plt.ylabel('OOF 예측 값 (Out-of-Fold Predicted)', fontsize=12)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # 3. 잔차(Residuals) 분포 확인 (OOF 기반)
    residuals = y_train - oof_preds
    plt.figure(figsize=(10, 6))
    sns.histplot(residuals, kde=True, bins=50)
    plt.title('잔차(실제-예측) 분포 (OOF 기반)', fontsize=16)
    plt.xlabel('잔차 (Residuals)', fontsize=12)
    plt.ylabel('빈도 (Frequency)', fontsize=12)
    plt.axvline(x=0, color='red', linestyle='--')
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    
    # 4. 학습 데이터와 테스트 예측 결과 분포 비교
    plt.figure(figsize=(10, 6))
    sns.kdeplot(y_train, label='학습 데이터 실제 값', color='blue', fill=True, alpha=0.5)
    sns.kdeplot(test_preds, label='테스트 데이터 예측 값', color='orange', fill=True, alpha=0.5)
    plt.title('학습 데이터와 테스트 예측의 분포 비교', fontsize=16)
    plt.xlabel('아파트 가격', fontsize=12)
    plt.ylabel('밀도 (Density)', fontsize=12)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # 5. SHAP 분석 (첫 번째 폴드 모델 사용)
    logger.write(">> SHAP 분석 시작 (계산에 시간이 다소 소요될 수 있습니다)...")
    try:
        explainer = shap.TreeExplainer(models[0])
        shap_sample = X_train_raw.sample(1000, random_state=42)
        shap_values = explainer.shap_values(shap_sample)

        logger.write(">> SHAP 요약 플롯 생성 중...")
        shap.summary_plot(shap_values, shap_sample, plot_type="dot", show=False)
        plt.title("SHAP 요약 플롯 (첫 번째 폴드 모델)", fontsize=16)
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        logger.write(f"[오류] SHAP 분석 중 문제가 발생했습니다: {e}", print_error=True)
    
    logger.write(">> [7단계 완료] 시각화 및 분석 완료.")

In [8]:
# --- 8. 최종 예측 및 제출 파일 생성 ---

if 'test_preds' in locals():
    logger.write(">> [8단계 시작] 최종 제출 파일 생성을 시작합니다...")
    try:
        # 최종 모델로 전체 데이터 학습 (나중에 다른 예측에 사용하기 위해 저장)
        logger.write(">> 전체 훈련 데이터로 최종 모델 학습 시작...")
        final_model = lgb.LGBMRegressor(device='cuda', **best_params)
        final_model.fit(X_train_raw, y_train)
        logger.write(">> 최종 모델 학습 완료.")
        
        # 모델 저장
        joblib.dump(final_model, MODEL_PATH)
        logger.write(f">> 모델 저장 완료: {MODEL_PATH}")

        # --- ✨ 제출 파일 생성 로직 ✨ ---
        logger.write(">> 'target' 컬럼만 포함된 제출 파일을 생성합니다.")
        submission_df = pd.DataFrame({'target': test_preds})
        submission_df['target'] = submission_df['target'].astype(int)
        
        # CSV 파일로 저장
        submission_df.to_csv(SUBMISSION_PATH, index=False)
        logger.write(f">> 제출 파일 생성 완료: {SUBMISSION_PATH}")
        
    except Exception as e:
        logger.write(f">> [오류] 제출 파일 생성 중 문제 발생: {e}", print_error=True)
    
    logger.write(">> 모델링 종료")
    logger.write("="*50 + "\n")
    logger.close()

2025-07-15 23:47:51 | >> [8단계 시작] 최종 제출 파일 생성을 시작합니다...
2025-07-15 23:47:51 | >> 전체 훈련 데이터로 최종 모델 학습 시작...
2025-07-15 23:51:28 | >> 최종 모델 학습 완료.
2025-07-15 23:51:28 | >> 모델 저장 완료: ../../model/price_prediction_model_1.pkl
2025-07-15 23:51:28 | >> 'target' 컬럼만 포함된 제출 파일을 생성합니다.
2025-07-15 23:51:28 | >> 제출 파일 생성 완료: ../../data/processed/submissions/price_prediction_submission.csv
2025-07-15 23:51:28 | >> 모델링 종료
2025-07-15 23:51:28 | ==================================================
